This is the second assignment for the Coursera course "Advanced Machine Learning and Signal Processing"


Just execute all cells one after the other and you are done - just note that in the last one you have to update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"

In [1]:
!wget https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20191228235615-0000
KERNEL_ID = 406cc56a-ff2c-4907-8f50-82da847eb57a
--2019-12-28 23:56:17--  https://github.com/IBM/coursera/raw/master/coursera_ml/a2.parquet
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet [following]
--2019-12-28 23:56:17--  https://raw.githubusercontent.com/IBM/coursera/master/coursera_ml/a2.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 59032 (58K) [application/octet-stream]
Saving to: 'a2.parquet'

100%[======================================>] 59,032      --.-K/s   in 0.002s  



Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [2]:
df=spark.read.load('a2.parquet')

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+-----------+-------------------+-------------------+-------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|
+-----+-----------+-------------------+-------------------+-------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|
|    0|34359738411| 26.786262674736566| -46.33193951911338| 20.880756008396055|
|    0| 8589934592|-16.203752396859194| 51.080957032176954| -96.80526656416971|
|    0|25769803852|   47.2048142440404| 

Please create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [12]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

Please instantiate a classifier from the SparkML package and assign it to the classifier variable. Make sure to either
1.	Rename the “CLASS” column to “label” or
2.	Specify the label-column correctly to be “CLASS”


In [13]:
df.printSchema()
# This renames the CLASS collum
df1=df.withColumnRenamed("CLASS","label")
df1.printSchema()

root
 |-- CLASS: integer (nullable = true)
 |-- SENSORID: long (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- Z: double (nullable = true)

root
 |-- label: integer (nullable = true)
 |-- SENSORID: long (nullable = true)
 |-- X: double (nullable = true)
 |-- Y: double (nullable = true)
 |-- Z: double (nullable = true)



In [30]:
#from pyspark.ml.classification import ###YOUR_CODE_GOES_HERE###"
from pyspark.ml.classification import MultilayerPerceptronClassifier

#classifier = ###YOUR_CODE_GOES_HERE###"

# specify layers for the neural network:
# input layer of size 3 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = [3, 50, 32, 2]

# create the trainer and set its parameters
classifier = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234, 
                                            labelCol="CLASS", featuresCol="features")

In [31]:
# Evaluating the pipeline steps
# 1 - VectorIndexing the class columm
vectorized = vectorAssembler.transform(df)
vectorized.show()

+-----+-----------+-------------------+-------------------+-------------------+--------------------+
|CLASS|   SENSORID|                  X|                  Y|                  Z|            features|
+-----+-----------+-------------------+-------------------+-------------------+--------------------+
|    0|         26| 380.66434005495194| -139.3470983812975|-247.93697521077704|[380.664340054951...|
|    0|         29| 104.74324299209692| -32.27421440203938|-25.105013725863852|[104.743242992096...|
|    0| 8589934658| 118.11469236129976| 45.916682927433534| -87.97203782706572|[118.114692361299...|
|    0|34359738398| 246.55394030642543|-0.6122810693132044|-398.18662513951506|[246.553940306425...|
|    0|17179869241|-190.32584900181487|  234.7849657520335|-206.34483804019288|[-190.32584900181...|
|    0|25769803830| 178.62396382387422| -47.07529438881511|  84.38310769821979|[178.623963823874...|
|    0|25769803831|  85.03128805189493|-4.3024316644854546|-1.1841857567516714|[85.03128805

Let’s train and evaluate…


In [32]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, classifier])

In [33]:
model = pipeline.fit(df)

In [34]:
prediction = model.transform(df)

In [44]:
prediction["CLASS","rawPrediction","probability","prediction"].show()

+-----+--------------------+--------------------+----------+
|CLASS|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+----------+
|    0|[11.4639676528196...|[0.99999999953130...|       0.0|
|    0|[4.81609706204504...|[0.99968184698923...|       0.0|
|    0|[11.6997765616922...|[0.99999999969080...|       0.0|
|    0|[15.7063238594170...|[0.99999999999993...|       0.0|
|    0|[7.30501161121329...|[0.99999911009648...|       0.0|
|    0|[12.7702228644763...|[0.99999999996806...|       0.0|
|    0|[5.86621536767274...|[0.99995963123884...|       0.0|
|    0|[11.1962166180986...|[0.99999999924363...|       0.0|
|    0|[4.60320520205290...|[0.99965458017012...|       0.0|
|    0|[10.5404364151770...|[0.99999999763313...|       0.0|
|    0|[7.26994588664018...|[0.99999832166696...|       0.0|
|    0|[12.6242923815326...|[0.99999999996937...|       0.0|
|    0|[11.7571774563319...|[0.99999999987634...|       0.0|
|    0|[4.20490340082175

In [36]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
binEval = MulticlassClassificationEvaluator().setMetricName("accuracy") .setPredictionCol("prediction").setLabelCol("CLASS")
    
binEval.evaluate(prediction) 

1.0

If you are happy with the result (I’m happy with > 0.55) please submit your solution to the grader by executing the following cells, please don’t forget to obtain an assignment submission token (secret) from the Coursera’s graders web page and paste it to the “secret” variable below, including your email address you’ve used for Coursera. (0.55 means that you are performing better than random guesses)


In [37]:
!rm -Rf a2_m2.json

In [38]:
prediction = prediction.repartition(1)
prediction.write.json('a2_m2.json')

In [39]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/IBM/coursera/master/rklib.py

--2019-12-29 00:18:04--  https://raw.githubusercontent.com/IBM/coursera/master/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2540 (2.5K) [text/plain]
Saving to: 'rklib.py'

100%[======================================>] 2,540       --.-K/s   in 0s      

2019-12-29 00:18:04 (28.3 MB/s) - 'rklib.py' saved [2540/2540]



In [40]:
import zipfile

def zipdir(path, ziph):
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

zipf = zipfile.ZipFile('a2_m2.json.zip', 'w', zipfile.ZIP_DEFLATED)
zipdir('a2_m2.json', zipf)
zipf.close()

In [41]:
!base64 a2_m2.json.zip > a2_m2.json.zip.base64

In [43]:
from rklib import submit
key = "J3sDL2J8EeiaXhILFWw2-g"
part = "G4P6f"
email = "giovani.machado@gmail.com"
secret = "K5wIf6vamYBSyOcP"

with open('a2_m2.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"LTL4F","id":"f_F-qCtuEei_fRLwaVDk3g~LTL4F~zzQp-CnQEequUgrP7atGQw","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
